In [1]:
import pandas as pd

df = pd.read_csv(f'results_supervised_old2.csv')
t = tuple(df.groupby(['zoo', 'clf', 'feature_type']))

column_names = {
    'lsb': r'\# LSB bits modified',
    'accuracy': r'Accuracy',
    'recall': r'Recall',
    'precision': r'Precision',
    'f1': r'F1-score',
}

df_chosen = t[0][1]
df_chosen.sort_values(by=['lsb'], inplace=True, ascending=[False])
df_chosen.drop(columns=['zoo', 'feature_type', 'clf', 'conf_mat'], inplace=True)
df_chosen.rename(columns=column_names, inplace=True)


precision_format = {k: "{:.0f}" if "lsb" in k.lower() else "{:.2f}" for k in column_names.values()}

tab = df_chosen.style.format_index(
    axis=1, formatter="{}".format).hide(
    axis=0).format(
        precision_format
    ).set_table_styles([
        {'selector': 'toprule', 'props': ':hline;'},
        {'selector': 'midrule', 'props': ':hline;'},
    ], overwrite=False)

prefix_str = r"""\begin{table}[th]
\centering
\begin{adjustbox}{width=\columnwidth}
"""

postfix_str = r"""
\end{adjustbox}
\end{table}
"""

tab_str=  tab.to_latex(column_format="|c c c c c|", hrules=False,)
tab_str_lines = tab_str.splitlines()
tab_str_lines = list(map(lambda line: line.replace(r'\\', r'\\ \hline'), tab_str_lines))

tab_str_processed = "\n".join(tab_str_lines)
final_tab_str = prefix_str+tab_str_processed+postfix_str

with open('table.tex', 'w') as f:
    f.write(final_tab_str)